<a href="https://colab.research.google.com/github/pmachadors/challengedados2/blob/main/semana3/semana3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip" && unzip semanas-3-e-4.zip -d dados/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 199 kB 59.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=382dc46aec719ab3b21a336c570e776e49352fff76a5eddf8cd3c602894fa015
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-12 12:35:11--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.93.28, 52.216.60.97, 52.217.46.116, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.93.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588308 (2.5M) [application/zip]
Saving to: ‘semanas-3-e-4.zip’

sema

# Criando sessão Spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName('semana3').getOrCreate()

df_imoveis = spark.read.parquet('/content/dados/dataset_ml_parquet')

df_imoveis.show(5)


+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

# Clustering com VectorAssembler

In [3]:
from pyspark.ml.feature import VectorAssembler

x = ['andar',
    'area_util',
    'banheiros',
    'quartos',
    'suites',
    'vaga',
    'condominio',
    'iptu',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul',
    'Academia',
    'Animais permitidos',
    'Churrasqueira',
    'Condomínio fechado',
    'Elevador',
    'Piscina',
    'Playground',
    'Portaria 24h',
    'Portão eletrônico',
    'Salão de festas']

vect_features = VectorAssembler(inputCols=x, outputCol='features')
imoveis_vect = vect_features.transform(df_imoveis)

imoveis_vect.show(5, truncate=False)

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------------------------------------------------------------------------------------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|features                                                                                        |
+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+---

# Padronização dos dados

In [4]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
imoveis_scaled = scaler.fit(imoveis_vect).transform(imoveis_vect)

imoveis_scaled.show(5, truncate=False)

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|

# Treinando PCA

In [5]:
from pyspark.ml.feature import PCA
k = len(x)

pca = PCA(k=k, inputCol='scaled_features',outputCol='pca_features')
model_pca = pca.fit(imoveis_scaled)
imoveis_pca = model_pca.transform(imoveis_scaled)

lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores



[0.277085377322873,
 0.4330261406218515,
 0.5235272089701289,
 0.580288999007754,
 0.6348779832271343,
 0.6828016505045259,
 0.7284955171033062,
 0.7719507672997754,
 0.8082013916221407,
 0.8366299175415439,
 0.8619471704531391,
 0.8823592010264175,
 0.9017740574586314,
 0.9179919432351559,
 0.932532283623452,
 0.946220072183121,
 0.9587579297383086,
 0.9705865473742374,
 0.9810516796158224,
 0.9907013282829159,
 0.9999999999999951,
 1.0]

In [6]:
import numpy as np

sum(np.array(lista_valores) <= 0.7)

6

# Redução de Dimencionalidade com PCA

In [7]:
from pyspark.ml.feature import PCA

pca2 = PCA(k=6, inputCol='scaled_features',outputCol='pca_features')
model_pca2 = pca2.fit(imoveis_scaled)
imoveis_pca2 = model_pca2.transform(imoveis_scaled)

imoveis_pca2.select('pca_features').show(5, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------+
|[-6.185066556186067,-1.5035861420376833,1.4021971813766747,-0.4627462052987944,0.05817557507559954,-2.06050203373801]    |
|[-3.2149125552925533,1.16996715944037,0.5164078437661357,3.2086930069839372,0.08994771500778509,1.1457808725760208]      |
|[-1.0068958653993225,1.4962100951692308,2.6741317795672224,0.02967168175495706,-0.043207662071430536,0.9916054175401698] |
|[-1.9528334788197674,2.054292371016602,-1.0129063602744157,-0.17794843497698848,-0.012607180989539779,0.2228967536340269]|
|[-0.3550159242190174,2.168729788605275,0.5165722841101543,2.7636650294554017,-0.06840160430886463,-0.09663724585553389]  |
+-------

In [8]:
sum(model_pca2.explainedVariance) * 100

68.2801650504526

# Criando Pipeline

In [9]:
from pyspark.ml import Pipeline

pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=x,outputCol='features'),\
                StandardScaler(inputCol='features', outputCol='scaled_features'),\
                PCA(k=6, inputCol='scaled_features',outputCol='pca_features')\
                ])

model_pipeline = pca_pipeline.fit(df_imoveis)
projection = model_pipeline.transform(df_imoveis)

projection.select('pca_features').show(5, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------+
|[-6.185066556186067,-1.5035861420376833,1.4021971813766747,-0.4627462052987944,0.05817557507559954,-2.06050203373801]    |
|[-3.2149125552925533,1.16996715944037,0.5164078437661357,3.2086930069839372,0.08994771500778509,1.1457808725760208]      |
|[-1.0068958653993225,1.4962100951692308,2.6741317795672224,0.02967168175495706,-0.043207662071430536,0.9916054175401698] |
|[-1.9528334788197674,2.054292371016602,-1.0129063602744157,-0.17794843497698848,-0.012607180989539779,0.2228967536340269]|
|[-0.3550159242190174,2.168729788605275,0.5165722841101543,2.7636650294554017,-0.06840160430886463,-0.09663724585553389]  |
+-------

# Clustering com K-Means

In [10]:
from pyspark.ml.clustering import KMeans

SEED = 1 

kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca',k=50, seed=SEED)
model_kmeans = kmeans.fit(projection)

projection_kmeans = model_kmeans.transform(projection)

projection_kmeans.select('pca_features','cluster_pca').show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------+-----------+
|pca_features                                                                                                             |cluster_pca|
+-------------------------------------------------------------------------------------------------------------------------+-----------+
|[-6.185066556186067,-1.5035861420376833,1.4021971813766747,-0.4627462052987944,0.05817557507559954,-2.06050203373801]    |18         |
|[-3.2149125552925533,1.16996715944037,0.5164078437661357,3.2086930069839372,0.08994771500778509,1.1457808725760208]      |10         |
|[-1.0068958653993225,1.4962100951692308,2.6741317795672224,0.02967168175495706,-0.043207662071430536,0.9916054175401698] |43         |
|[-1.9528334788197674,2.054292371016602,-1.0129063602744157,-0.17794843497698848,-0.012607180989539779,0.2228967536340269]|20         |
|[-0.3550159242190174,2.168729788605275,0.516572

# Testando K-means

In [11]:
import plotly.express as px
from pyspark.ml.functions import vector_to_array

projection_kmeans = projection_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                     .withColumn('y', vector_to_array('pca_features')[1])
                            

fig = px.scatter(projection_kmeans.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['valor'])
fig.show()


# Filtrando imoveis

In [47]:
id = projection_kmeans.collect()[0]['id']
cluster = projection_kmeans.select('cluster_pca').where(f.col('id') == id).collect()[0][0]

imoveis_recomendados = projection_kmeans.select('id','pca_features').where(f.col('cluster_pca') == cluster)
componentes_imoveis = imoveis_recomendados.select('pca_features').where(f.col('id') == imoveis_recomendados.id).collect()[0][0]
componentes_imoveis

DenseVector([-6.1851, -1.5036, 1.4022, -0.4627, 0.0582, -2.0605])

# Calculando distância Euclidiana

In [48]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType

def calcula_distance(value):
  return euclidean(componentes_imoveis, value)

udf_calcula_distance = f.udf(calcula_distance,FloatType())

<function __main__.calcula_distance(value)>

In [31]:
imoveis_recomendados_dist = imoveis_recomendados.withColumn('Dist', udf_calcula_distance('pca_features'))
recomendados = spark.createDataFrame(imoveis_recomendados_dist.orderBy('Dist').take(10))
recomendados.select('id','Dist').show(truncate=False)



+------------------------------------+--------------------+
|id                                  |Dist                |
+------------------------------------+--------------------+
|00002dd9-cc74-4809-b5a5-850adf0e7526|0.0                 |
|33aeb3e6-f85d-49b0-a758-c5252c5623e6|0.01103171519935131 |
|842a87a0-7602-4b86-8969-75fee442fabf|0.016404559835791588|
|5dac977a-6cfb-4283-a247-9478beca6bce|0.016408812254667282|
|a1c430e5-fe77-4b0f-a08c-001f57f11f89|0.021944619715213776|
|d4a64276-c147-4b39-b5ce-19d03fbdb744|0.033049363642930984|
|a99baaa1-00ef-4906-8903-3f8a38a06385|0.04117828980088234 |
|b3d1d649-f026-48ac-a0a5-dbb14ada1f61|0.043229375034570694|
|5bdbf3b3-fc45-4387-84f0-3c0cc3b1580c|0.05620022863149643 |
|1238503b-dbbc-4664-a48b-e94195624404|0.058948028832674026|
+------------------------------------+--------------------+



# Criando função recomendadora

In [53]:
from pyspark.sql.types import FloatType

def calcula_distance(value):
    return euclidean(componentes_imoveis, value)

def recomendador(id_imovel):
  cluster = projection_kmeans.select('cluster_pca').where(f.col('id') == id_imovel).collect()[0][0]
  imoveis_recomendados = projection_kmeans.where(projection_kmeans.cluster_pca == cluster)\
                                       .select('id', 'pca_features')

  componentes_imoveis = imoveis_recomendados.where(imoveis_recomendados.id == id_imovel)\
                                          .select('pca_features').collect()[0][0]

  udf_calcula_distance = f.udf(calcula_distance,FloatType())
  imoveis_recomendados_dist = imoveis_recomendados.withColumn('dist', udf_calcula_distance('pca_features'))
  recomendados = spark.createDataFrame(imoveis_recomendados_dist.orderBy('dist').take(10))
  recomendados.select('id','dist').show(truncate=False)

  





In [54]:
recomendador('1238503b-dbbc-4664-a48b-e94195624404')

+------------------------------------+--------------------+
|id                                  |dist                |
+------------------------------------+--------------------+
|00002dd9-cc74-4809-b5a5-850adf0e7526|0.0                 |
|33aeb3e6-f85d-49b0-a758-c5252c5623e6|0.01103171519935131 |
|842a87a0-7602-4b86-8969-75fee442fabf|0.016404559835791588|
|5dac977a-6cfb-4283-a247-9478beca6bce|0.016408812254667282|
|a1c430e5-fe77-4b0f-a08c-001f57f11f89|0.021944619715213776|
|d4a64276-c147-4b39-b5ce-19d03fbdb744|0.033049363642930984|
|a99baaa1-00ef-4906-8903-3f8a38a06385|0.04117828980088234 |
|b3d1d649-f026-48ac-a0a5-dbb14ada1f61|0.043229375034570694|
|5bdbf3b3-fc45-4387-84f0-3c0cc3b1580c|0.05620022863149643 |
|1238503b-dbbc-4664-a48b-e94195624404|0.058948028832674026|
+------------------------------------+--------------------+

